In [1]:
import pandas as pd
from langdetect import detect
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import numpy as np
import sys
import os
import time
from nltk.tag import pos_tag

In [ ]:
target = pd.read_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1_Targets.csv",
                    sep = ',',
                     skiprows=range(112,113), # 112 wadliwy wiersz
                    names=['TableID', 'ColumnID', 'RowID'])
target[:3]

In [9]:
start = time.time()

# słownik, z którego będzie tworozny wynikowy df
answers_dict = {"TableID":[],
               "ColumnID":[],
               "RowID":[],
               "URI":[]}

# zmienna, aby ograniczyć liczbę przetwarzanych wierszy, niżej masz 3 linijki skomentowane,
# odkomentowujesz je i ustawiasz sobię liczbę wierszy w tym warunku, aktualnie 1000
counter = 0
# zlicza ile linków nie zostało znalezionych w ogóle,
# tzn. po wysłaniu zapytania nie dostaliśmy żądnej odpwiedzi od sparql (wiersze oznaczone przez NOT FOUND URI)
not_found_uris = 0

for index, row in target.iterrows():
    try:
#         counter = counter + 1
#         if counter == 1000:
#             break


        table_id = row['TableID']
        column_id = row['ColumnID']
        row_id = row['RowID']
        
        # wczytanie odpowiedniej tabeli
        df = pd.read_csv(f"D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1\{table_id}.csv",
                    sep=',')
        
        # wyciągniecie wartości komórki, która nas interesuje
        cell_content = df.iloc[row_id-1:row_id, column_id:(column_id+1)].values[0][0]
        
        # formatowanie tej zawartości, aby znaleźć jak najwięćej linków
        # przyglądałem się komórkom, dla których nie mogłem znaleźć linku i widziałem jakieś dziwne znaki,
        # czy inne rzeczy i prórowałem je naprawiać
        cell_content = cell_content.replace('-', ' ')
        cell_content = cell_content.replace(' The', '')
        cell_content = cell_content.replace(',', '')
        cell_content = cell_content.replace('[', '')
        cell_content = cell_content.replace(']', '')
        cell_content = cell_content.replace('?', '')
        
        # usunięcie białych znaków z początku i końca stringa
        cell_content = cell_content.strip()

        # aktualnie nie używane, ale pewnie będzie
        # pos_tag() - kategoryzuje słowa, np. znajduje czasowniki, rzeczowniki, nazwy własne (jak imiona i nazwiska)
        # chodzi o to, że czasami odpowiedzi są w angielskiej dbpedii,
        # np. Alex Rodriguez wykrywa jako język ca - pewnie kataloński, 
        # ale nie ma jego odpowiednika w katalońskiej dbpedii, a w angielskiej jest
        # dlatego chcę wykrywać nazwy wkłasne jak imiona i nazwiska i wtedy zawsze szukać w angielksiej dbpedii,
        # ale z tym też był problem czasami i na razie nie myślałem nad tym dalej
        # NNP - to kategoria nazwy własnej, np. imię
#         tags = pos_tag(cell_content.split())
#         is_one_NNP = False
#         for tag in tags:
#             if tag[1] == "NNP":
#                 is_one_NNP = True
#                 break
#         try:
#             if is_one_NNP:
#                 lang = '@en'
#             else:
#                 lang = str('@') + str(detect(cell_content))
#         except Exception:
#             lang = ''
        
        # pierwsze zapytanie - bierze zawartość sformatowanej komórki i wyszukuje linku,
        # na razie jest ustawiona na stałe angielska dbpedia (#en) z powodu tego co wcześniej opisałem
        query = """SELECT ?resource WHERE { ?resource rdfs:label """ + '"' + str(cell_content) + '"' + "@en" + '}'
        
        # zapytanie ignorujące wielkość liter, ale wtedy wykonuje się baaaardzo długo...
#         query = "SELECT ?resource WHERE { ?resource rdfs:label ?o " + "FILTER regex(?o, \"^" + cell_content + "$\", \"i\")" + '}'
        
    
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        # ten warunek sprawdza czy znaleziono jakiekolwiek linki, jeśli tak to omija i przechodzi dalej
        # jeśli jednak nic nie znalazł to wykonuje na wartości komórki cell_content.capitalize()
        # zmienia to np. TREE SWALLOW na Tree swallow,
        # dbpedia traktuje te dwie rzeczy jak coś różnego, wielkość liter ma znaczenie
        # i ostatnie 2 linijki znowu wykonują zapytanie ze zmienionym cell_content
        if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
            cell_content_temp = cell_content.capitalize() # TREE -> Tree
            query = """SELECT ?resource WHERE { ?resource rdfs:label """ + '"' + str(cell_content_temp) + '"' + "@en" + '}'
            sparql.setQuery(query)
            results = sparql.query().convert()

        # jeśli nadal nic nie znaleziono no to trudno, nie mamy linku, wstawiamy NOT FOUND URI
        if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
            answers_dict['TableID'].append(table_id)
            answers_dict['ColumnID'].append(column_id)
            answers_dict['RowID'].append(row_id)
            answers_dict['URI'].append("NOT FOUND URI")
            # printy przydatne do sprawdzania jakich zawartości nam nie wyszukał,
            # trzeba analizowac i dodawać kolejne warunki, zmieniać cell_content na różne sposoby
#             print(table_id, column_id, row_id)
#             print(cell_content)
#             print(query)
            not_found_uris = not_found_uris + 1
            continue

        # jeśli jednak mamy jakies linki to wyciągamy go z jsona,
        # linków może być więcej niż 1, aktualnie bierzemy pierwszy lepszy
        results_list = results['results']['bindings']
        for result_dict in results_list:
    #         print(result_dict)
            uri = result_dict['resource']['value']
            match = re.match("http://dbpedia.org/resource/*", uri)
            
            # czasami pojawiają dziwne linki np. http://dbpedia.org/resource/Category:Max_Weber
            # my takich nie chcemy i musimy ich unikać
            # interesują nas tylko np.  http://dbpedia.org/resource/Max_Weber
            # więc to jest warunek pozbywajacy się wszystkich linków z Category:
            if match and uri[5:].find("Category:") == -1:
                answers_dict['TableID'].append(table_id)
                answers_dict['ColumnID'].append(column_id)
                answers_dict['RowID'].append(row_id)
                answers_dict['URI'].append(uri)
                break
    except Exception as e:
        # na początku były problemy z danymi, naprawiłem
        # potem zaczęły pojawiać sę problemy, żę serwer czasami nie odpowiadał,
        # więc jak jest error to olewamy i przechodzimy do następnego wiersza
        # będzie trzeba zrobić, że jak serwer nie odpowie z jakiegoś powodu to trzeba ponowić to samo zapytanie
        # zamiast pomijać, bo tracimy wiersze
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(table_id, column_id, row_id) # jaki plik, kolumna i wiersz
        print(cell_content) # zawartość komórki
        print(e) # jaki błąd, teraz chyba tylko z brakiem połączenia z serwerem
        print(exc_tb.tb_lineno) # w jakiej liniii błąd
        continue



# print(answers_dict)
result_df = pd.DataFrame(answers_dict)
end = time.time()
print(end-start)
print(errors)

46671561_0_6122315295162029872 0 166
Tom Clancys Rainbow Six Vegas 2
<urlopen error [WinError 10060] Próba połączenia nie powiodła się, ponieważ połączona strona nie odpowiedziała poprawnie po ustalonym okresie czasu lub utworzone połączenie nie powiodło się, ponieważ połączony host nie odpowiedział>
68
13719111_1_5719401842463579519 0 0
Amarcord
index 0 is out of bounds for axis 0 with size 0
24
33401079_0_9127583903019856402 0 60
Karl von Frisch
<urlopen error [WinError 10060] Próba połączenia nie powiodła się, ponieważ połączona strona nie odpowiedziała poprawnie po ustalonym okresie czasu lub utworzone połączenie nie powiodło się, ponieważ połączony host nie odpowiedział>
56
69881946_0_1105130426898457358 1 42
From Here to Eternity
<urlopen error [WinError 10060] Próba połączenia nie powiodła się, ponieważ połączona strona nie odpowiedziała poprawnie po ustalonym okresie czasu lub utworzone połączenie nie powiodło się, ponieważ połączony host nie odpowiedział>
56
90196673_0_5458330

In [12]:
print(result_df.shape)
print(target.shape)

(8048, 4)
(8417, 3)


In [14]:
# zapisanie submission
# przez evaluator codes można sprawdzić wyniki
result_df.to_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\evaluator_codes\evaluator_codes\data\my_submission.csv",
                 index=False,
                 sep=',',
                header=False)

### NIEISTOTNE, TESTY ITP.

In [ ]:
df = pd.read_csv("D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\CEA_Round1\\13719111_1_5719401842463579519.csv",
                sep=',')
df[:3]

In [ ]:
detect(df.iloc[0:1, 1:2].values[0][0])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Norway> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
for result in results["results"]["bindings"]:
    print(result["label"]["value"])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ont: <http://dbpedia.org/ontology/>
    SELECT ?a, ?c
    WHERE { 
            ?a ont:location <http://dbpedia.org/resource/Delft> . 
            FILTER regex(?a, "Ikea", "i")
          }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
# print(result['a'])
# print(result['b'])
# print(result['c'])

# for result in results["results"]["bindings"]:
#     print(type(result))

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {e}
    VALUES (?r) {e} ({source}) {w}
        {e} ?r ?p ?o {w}
        UNION
        {e} ?s ?r ?o {w}
        UNION
        {e} ?s ?p ?r {w}
    {w} 
'''.format(source=sss, e="{", w="}"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {
    VALUES (?r) { (dbr:Gra_o_tron) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
    } 
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)



In [ ]:
SELECT ?r ?p ?o
WHERE {
 VALUES (?r) { (dbr:Cat) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
}

select ?s
where {
        { ?s rdfs:label "Kot domowy"@pl }
        UNION
        { "Kot domowy"@pl rdfs:label ?s }
} 

SELECT ?r
WHERE {
        { ?r rdfs:label "Kaspisches Meer"@de }

}

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?resource
    WHERE { ?resource rdfs:label "Kaspisches Meer"@de }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# print(results['results']['bindings'][1]['r']['value'])
# results = str(results['results']['bindings'])
print(str(results['results']['bindings']))
if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
    print("nie ma")
else:
    print('jest')
# for result_dict in results_list:
#     uri = result_dict['resource']['value']
#     match = re.match("http://dbpedia.org/resource/*", uri)
#     if match:
#         print(uri)
#     else:
#         print("NIE")

# print(results['results']['bindings'])

In [ ]:
import re
match = re.match("http://dbpedia.org/resource/*",'http://dbpedia.org/resource/Caspian_Sea', re.M|re.I)
if match:
    print(match.group())